# Save output

In [8]:
using Statistics
include("src-fig/plotting.jl")
foldername = "../scratch/turbulence-at-fronts/Strain-Large"

slicefoldername = joinpath(foldername, "slices")
OUTPUT = joinpath(foldername, "output.jld2")

mkpath(slicefoldername)

save_slices(joinpath(slicefoldername, "u.jld2"), OUTPUT, "u")
save_slices(joinpath(slicefoldername, "v.jld2"), OUTPUT, "v")
save_slices(joinpath(slicefoldername, "w.jld2"), OUTPUT, "w")
save_slices(joinpath(slicefoldername, "b.jld2"), OUTPUT, "b")

800/800
800/800
800/800
800/800


# 3D Schematic of the simulation

In [ ]:
foldername = "slices"

iterations, times = iterations_times(foldername)
sp = simulation_parameters(foldername)
xsᶜ, xsᶠ, ysᶜ, ysᶠ, zsᶜ, zsᶠ = grid_nodes(foldername)
inds = centre_indices(foldername)

#U = [-sp.\alpha * x for x in xs\^f, y in 1:1, z in 1:1]

u_top = get_field(joinpath(foldername, "u.jld2"), "top")
u_south = get_field(joinpath(foldername, "u.jld2"), "south")
u_east = get_field(joinpath(foldername, "u.jld2"), "east")

b_top = get_field(joinpath(foldername, "u.jld2"), "top")
b_south = get_field(joinpath(foldername, "u.jld2"), "south")
b_east = get_field(joinpath(foldername, "u.jld2"), "east")

fig = Figure()

ax = Axis3(fig[1, 1];
    limits=(-sp.Lh/2000, sp.Lh/2000, -sp.Ly/2000, sp.Ly/2000, -sp.Lz, sp.Lz/4)
)

umax = mapreduce(max, [u_top, u_south, u_east]) do u
    maximum(abs, u)
end

colorrange = (-umax, umax)
colormap = :balance

# Plot the heatmap of u
heatmap!(ax, xsᶠ ./ 1000, ysᶜ ./ 1000, u_top; colorrange, colormap, transformation=(:xy, 0))
heatmap!(ax, xsᶠ ./ 1000, zsᶜ, u_south; colorrange, colormap, transformation=(:xz, ysᶜ[1] / 1000))
heatmap!(ax, ysᶜ ./ 1000, zsᶜ, u_south; colorrange, colormap, transformation=(:yz, sp.Lh/2000))

# Represent the stretched xs with linesegments

# Plot the buoyancy contours
contour!(ax, xsᶠ ./ 1000, ysᶜ ./ 1000, b_top; color=(:black, 1.0), transformation=(:xy, 0))
contour!(ax, xsᶠ ./ 1000, zsᶜ, b_south; color=(:black, 1.0), transformation=(:xz, ysᶜ[1] / 1000))
contour!(ax, ysᶜ ./ 1000, zsᶜ, b_south; color=(:black, 1.0), transformation=(:yz, sp.Lh/2000))

# Plot some cooling stinklines
cooling_x = range(-sp.Lh/2000, sp.Lh/2000, 10)
cooling_y = range(-sp.Ly/2000, sp.Ly/2000, 3)
cooling_z = range(0, sp.Lz / 4, 50)
for x in cooling_x, y in cooling_y
    xs = x .+ 0.1 * sin.(cooling_z)
    ys = y .+ 0.1 * sin.(cooling_z)
    lines!(ax, xs, ys, cooling_z; color=(:red, 0.5))
end

# Plot some background flow arrows
arrows_x = range(-sp.Lh/2000, sp.Lh/2000, 10)
arrows_y = range(-sp.Ly/2000, sp.Ly/2000, 3)
arrows_z = sp.Lz / 4

U = [-x for x in arrows_x, y in arrows_y]
V = [y for x in arrows_x, y in arrows_y]
W = [0 for x in arrows_x, y in arrows_y]

arrows!(ax, arrows_x, arrows_y, arrows_z, U, V, W)

fig